# Main notebook

- DCGAN Adapted from [this tutorial](https://keras.io/examples/generative/dcgan_overriding_train_step/)
- VAE adapted from this [tutorial](https://keras.io/examples/generative/vae/#train-the-vae)

## Setup

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import gdown
from zipfile import ZipFile
from pathlib import Path
import time
import os

In [2]:
import sys
def is_in_collab_env():
    return 'google.colab' in sys.modules

if is_in_collab_env():
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT = Path("/content/drive/MyDrive/INF8225/notebooks")
else:
    ROOT = Path('..').resolve()

ROOT_DATA_PATH = ROOT/"data"
ROOT_MODEL_PATH = ROOT/"models"
ROOT_DATA_PATH.mkdir(parents=True, exist_ok=True)
ROOT_MODEL_PATH.mkdir(parents=True, exist_ok=True)

Mounted at /content/drive


In [3]:
from enum import Enum
class Dataset_names(str, Enum):
    CelebA = "CelebA"
    Anime_faces = 'Anime_faces'

class Model_names(str, Enum):
    GAN = "GAN"
    VAE = 'VAE'

def get_model_dir(dataset_name: Dataset_names, model_name: Model_names):
    path = ROOT_MODEL_PATH/f'{model_name}-{dataset_name}'
    path.mkdir(parents=True, exist_ok=True)
    return path

def get_data_dir(dataset_name: Dataset_names, model_name: Model_names):
    path = ROOT_DATA_PATH/f'{model_name}-{dataset_name}'
    path.mkdir(parents=True, exist_ok=True)
    return path


Selects the dataset and the model that will be used.

In [4]:
# Select dataset + model
dataset_name=Dataset_names.CelebA
model_name=Model_names.VAE

# Get/create their directory
MODEL_PATH = get_model_dir(dataset_name, model_name)
DATA_PATH = get_data_dir(dataset_name, model_name)

## Datasets

### CelebA

In [5]:
def get_celebA_dataset():
    ## Download dataset
    os.makedirs("celeba_gan", exist_ok=True)

    zip_path = ROOT_DATA_PATH/'celebA'/'img_align_celeba.zip'
    if not zip_path.exists():      
        url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
        gdown.download(url, zip_path, quiet=True)

    img_path = 'celeba_gan'
    with ZipFile(zip_path, "r") as zipobj:
        zipobj.extractall(img_path)

    ## Build keras dataset object
    dataset = keras.preprocessing.image_dataset_from_directory(
        img_path, label_mode=None, image_size=(64, 64), batch_size=32, seed=42
    )
    ## Preprocessing
    dataset = dataset.map(lambda x: x / 255.0)
    return dataset

### Anime Faces

In [6]:
def get_Anime_dataset():
    ## Download dataset
    os.makedirs("gan_anime", exist_ok=True)
    url = "https://drive.google.com/uc?id=1w9c2-Nx3Db9p03F2Dy4RWw9kOQkE3gCV"
    output = "gan_anime/archive_anime.zip"
    gdown.download(url, output, quiet=True)
    with ZipFile("gan_anime/archive_anime.zip", "r") as zipobj:
        zipobj.extractall("gan_anime")

    dataset = keras.preprocessing.image_dataset_from_directory(
        "gan_anime/images", label_mode=None, image_size=(64, 64), batch_size=32, seed=42
    )
    ## Preprocessing
    dataset = dataset.map(lambda x: x / 255.0)


    return  dataset

### Load dataset

In [7]:
def plot_multiple_images(dataset):
    '''
    Plot multiple images in a grid
    '''
    fig, axes = plt.subplots(5,5, figsize = (14,14))
    sample = dataset.unbatch().take(25)
    sample = [image for image in sample]

    idx = 0
    for row in range(5):
        for column in range(5):
            axes[row, column].imshow(sample[idx])
            idx+=1
    return

In [8]:
# https://towardsdatascience.com/how-to-split-a-tensorflow-dataset-into-train-validation-and-test-sets-526c8dd29438
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1

    ds = ds.shuffle(10000, seed=42)

    ds_size = tf.data.experimental.cardinality(ds).numpy()
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
_dataset = None
if dataset_name == Dataset_names.CelebA:
    _dataset = get_celebA_dataset()
else:
    _dataset = get_Anime_dataset()
    
train, valid, test = get_dataset_partitions_tf(_dataset)
plot_multiple_images(_dataset)

## Architectures

### DCGAN

In [ ]:
class GAN(keras.Model):
    def __init__(self, latent_dim):
        super(GAN, self).__init__()
        self.latent_dim = latent_dim
        self.discriminator = self.__get_discriminator()
        self.generator = self.__get_generator()

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    def get_generator(self):
        return self.generator

    def __get_discriminator(self):
        return keras.Sequential(
                [
                    keras.Input(shape=(64, 64, 3)),
                    layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
                    layers.LeakyReLU(alpha=0.2),
                    layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
                    layers.LeakyReLU(alpha=0.2),
                    layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
                    layers.LeakyReLU(alpha=0.2),
                    layers.Flatten(),
                    layers.Dropout(0.2),
                    layers.Dense(1, activation="sigmoid"),
                ],
                name="discriminator",
            )
    def __get_generator(self):
        return keras.Sequential(
            [
                keras.Input(shape=(self.latent_dim,)),
                layers.Dense(8 * 8 * 128),
                layers.Reshape((8, 8, 128)),
                layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
                layers.LeakyReLU(alpha=0.2),
                layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
                layers.LeakyReLU(alpha=0.2),
                layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
                layers.LeakyReLU(alpha=0.2),
                layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
            ],
            name="generator",
        )
    
    def generate_samples(self, num_img):
        """
        Generates num_img images.
            Returns:
                imgs: an array of PIL images
        """
        random_latent_vectors = tf.random.normal(shape=(num_img, self.latent_dim))
        generated_images = self.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imgs = [ keras.preprocessing.image.array_to_img(generated_image) for generated_image in generated_images]
        return imgs

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


### VAE


In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
class VAE(keras.Model):
    def __init__(self, latent_dim, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.latent_dim = latent_dim
        self.encoder = self.__get_encoder()
        self.decoder = self.__get_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def get_generator(self):
        return self.decoder

    def __get_encoder(self):
        encoder_inputs = keras.Input(shape=(64, 64, 3)) # avant 28, 28, 1
        x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
        x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(self.latent_dim, activation="relu")(x)
        z_mean = layers.Dense(self.latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(self.latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        return keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    def __get_decoder(self):
        latent_inputs = keras.Input(shape=(self.latent_dim,))
        x = layers.Dense(16 * 16 * 64, activation="relu")(latent_inputs) # avant: 7 * 7 * 64
        x = layers.Reshape((16, 16, 64))(x) # avant: 7,7,64
        x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(rate=0.25)(x)
        x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(rate=0.25)(x)
        decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
        return keras.Model(latent_inputs, decoder_outputs, name="decoder")

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def predict(self, input):
        z_mean, z_log, z = self.encoder.predict(input)
        return self.decoder.predict(z)
    
    def generate_samples(self, num_img):
        """
        Generates num_img images.
            Returns:
                imgs: an array of PIL images
        """
        random_latent_vectors = tf.random.normal(shape=(num_img, self.latent_dim))
        generated_images = self.decoder(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imgs = [ keras.preprocessing.image.array_to_img(generated_image) for generated_image in generated_images]
        return imgs

## Training code

### Callbacks

In [ ]:
class SaveImagesCallback(keras.callbacks.Callback):
    def __init__(self, dir_path, latent_vectors, num_img=3, epoch_offset=0):
        self.num_img = num_img
        self.epoch_offset = epoch_offset
        self.dir_path = dir_path
        self.latent_vectors = latent_vectors

    def on_epoch_end(self, epoch, logs=None):
        epoch_adjusted = self.epoch_offset+epoch
        if (epoch_adjusted % 10) == 0:
            
            generator = self.model.get_generator()
            generated_images = generator(self.latent_vectors)
            generated_images *= 255
            generated_images.numpy()
            
            dir_path = self.dir_path/f'e:{epoch_adjusted}'
            dir_path.mkdir(parents=True, exist_ok=True)
            crnt_timestamp = str(time.time()).split('.')[0]
            for i in range(len(self.latent_vectors)):
                img = keras.preprocessing.image.array_to_img(generated_images[i])
                img.save(dir_path/f'generated_img_e:{epoch_adjusted}_{i}_t:{crnt_timestamp}.png')


In [ ]:
class SaveImagesGifCallback(keras.callbacks.Callback):
    '''
    Saves 25 images in 1 image after each epoch
    '''
    def __init__(self, dir_path, num_img=3, latent_dim=128, epoch_offset=0):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.epoch_offset = epoch_offset
        self.dir_path = dir_path
        self.random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))

    def on_epoch_end(self, epoch, logs=None):

        epoch_adjusted = self.epoch_offset+epoch
        if (epoch_adjusted % 1) == 0:
            random_latent_vectors = self.random_latent_vectors
            generator = self.model.get_generator()

            generated_images = model.generate_samples(self.num_img)


            fig, axes = plt.subplots(5,5, figsize = (14,14))
            idx = 0
            for row in range(5):
                for column in range(5):
                    image = generated_images[idx] #* 255
                    #image = image.astype("int32")
                    axes[row, column].imshow(image)
                    axes[row, column].axis("off")
                    idx+=1
            
            dir_path = self.dir_path  #f'e:{epoch_adjusted}'
            dir_path.mkdir(parents=True, exist_ok=True)
            crnt_timestamp = str(time.time()).split('.')[0]
            
            plt.savefig(dir_path/f'Epoch_{epoch_adjusted}_timestamp_{crnt_timestamp}.jpg')
            plt.close()


### Training

In [ ]:
def get_GAN(latent_dim, load_weights= False, weights_path=None):
    """
    Parameters:
        latent_dim: The size of the latent dim
        load_weights: When set to true, loads weights
        weights_path: The path to the weights
    Returns:
        gan
    """
    gan = GAN(latent_dim)
    gan.compile(
        d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss_fn=keras.losses.BinaryCrossentropy(),
    )
    if load_weights:
        gan.load_weights(weights_path)
    return gan

def get_VAE(latent_dim, load_weights= False, weights_path=None):
    """
    Parameters:
        latent_dim: The size of the latent dim
        load_weights: When set to true, loads weights
        weights_path: The path to the weights
    Returns:
        vae
    """
    R_LOSS_FACTOR = 10000
    vae = VAE(latent_dim)
    vae.compile(keras.optimizers.Adam(learning_rate=0.0005), R_LOSS_FACTOR, metrics=["val_loss"])
    if load_weights:
        vae.load_weights(weights_path)
    return vae

In [ ]:
## Model configuration
load_weights = False
weights_path = None
latent_dim = 200

## Training specifics
epochs = 30 # In practice, use ~100 epochs

In [ ]:
model = None
if model_name == Model_names.GAN:
    model = get_GAN(latent_dim, load_weights, weights_path)
else:
    model = get_VAE(latent_dim, load_weights, weights_path)

In [ ]:
num_img=10
latent_vectors = tf.random.normal(shape=(num_img, latent_dim), seed=42)

save_images_callback = SaveImagesCallback(dir_path=DATA_PATH, latent_vectors=latent_vectors, num_img=10)
# save_gif_images_callback = SaveImagesGifCallback(dir_path=DATA_PATH, num_img=30, latent_dim=latent_dim)
mcp_save = tf.keras.callbacks.ModelCheckpoint(MODEL_PATH/'e:{epoch:02d}',period=10, save_weights_only=True)

model.fit(
    train, epochs=epochs, callbacks=[save_images_callback, mcp_save ], validation_data=valid
)

Some of the last generated images around epoch 30
(results keep improving after that):

![results](https://i.imgur.com/h5MtQZ7l.png)